In [42]:
import numpy as np
import os
import boto3
from google.protobuf.json_format import MessageToDict
import json

In [36]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [4]:
bucket = 'demo-saeed'
prefix = 'fraudcredit-keras'
dataset_name = 'creditcard.csv'

In [95]:
import pandas as pd
df = pd.read_csv('data/creditcard.csv', sep=',',  nrows=100)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [96]:
from sklearn.preprocessing import StandardScaler

data = df.drop(['Time'], axis=1)

data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

In [97]:
df30 = data
df30.to_csv('data/creditcard30_train.csv',index=None)
df30.drop(['Class'], axis=1).to_csv('data/creditcard30_test.csv',index=False,header=False)

In [22]:
def concatdf_ntimes(df,n):
    tmp = df
    for i in range(n):
        if i%100==0:
            print(i)
        a = df.iloc[:,:-1]  
        tmp = pd.concat([a, tmp], axis=1)
    return tmp

In [23]:
df150 = concatdf_ntimes(data,6)
df150.to_csv('data/creditcard180_train.csv',index=None)
df150.drop(['Class'], axis=1).to_csv('data/creditcard180_test.csv',index=False,header=False)

0


In [25]:
df15k = concatdf_ntimes(data,600)
df15k.to_csv('data/creditcard18k_train.csv',index=None)
df15k.drop(['Class'], axis=1).to_csv('data/creditcard18k_test.csv',index=False,header=False)

0
100
200
300
400
500


In [26]:
df150k = concatdf_ntimes(df15k,10)
df150k.to_csv('data/creditcard180k_train.csv',index=None)
df150k.drop(['Class'], axis=1).to_csv('data/creditcard180k_test.csv',index=False,header=False)

0


In [27]:
df1_5m = concatdf_ntimes(df150k,10)
df1_5m.shape
df1_5m.to_csv('data/creditcard1_8m_train.csv',index=None)
df1_5m.drop(['Class'], axis=1).to_csv('data/creditcard1_8m_test.csv',index=None,header=False)

0


# Convert to RecordIO

In [28]:
from sagemaker.amazon.common import write_numpy_to_dense_tensor

In [1]:
data.values

NameError: name 'data' is not defined

In [49]:
df1_5m.shape

(100, 2108910)

In [50]:
import sagemaker.amazon.common as smac

# vectors = np.array([t.tolist() for t in train_set[0]]).astype('float32')
# labels = np.where(np.array([t.tolist() for t in train_set[1]]) == 0, 1.0, 0.0).astype('float32')

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, df1_5m.values, labels=None)
buf.seek(0)

0

In [51]:
import shutil

buf.seek(0)
with open('data/df1_5m.ext', 'wb') as f:
    shutil.copyfileobj(buf, f, length=131072)

In [1]:
!aws s3 ls s3://demo-saeed/iris

                           PRE iris/


# Convert to TFrecord

In [56]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [58]:
!ls -al data/

total 11333556
drwxrwxr-x  3 ec2-user ec2-user       4096 Mar  9 20:07 .
drwxrwxr-x 10 ec2-user ec2-user       4096 Mar 10 00:08 ..
-rw-rw-r--  1 ec2-user ec2-user  365171807 Mar  8 01:03 creditcard180k_test.csv
-rw-rw-r--  1 ec2-user ec2-user   76691500 Mar  9 20:07 creditcard180k_test.tfrecords
-rw-rw-r--  1 ec2-user ec2-user  365899223 Mar  8 01:02 creditcard180k_train.csv
-rw-rw-r--  1 ec2-user ec2-user     386659 Mar  8 01:01 creditcard180_test.csv
-rw-rw-r--  1 ec2-user ec2-user      83754 Mar  8 22:19 creditcard180_test.tfrecords
-rw-rw-r--  1 ec2-user ec2-user     387635 Mar  8 01:01 creditcard180_train.csv
-rw-rw-r--  1 ec2-user ec2-user   33197437 Mar  8 01:01 creditcard18k_test.csv
-rw-rw-r--  1 ec2-user ec2-user    6975500 Mar  9 20:06 creditcard18k_test.tfrecords
-rw-rw-r--  1 ec2-user ec2-user   33263753 Mar  8 01:01 creditcard18k_train.csv
-rw-rw-r--  1 ec2-user ec2-user 4016889877 Mar  8 01:19 creditcard1_8m_test.csv
-rw-rw-r--  1 ec2-user ec2-user  843568000 Mar  9 19:

In [59]:
import tensorflow as tf
filepath = os.path.join('data/creditcard1_8m_test.csv')
print('Reading', filepath)
tf_filename = 'data/creditcard1_8m_test.tfrecords'
print('Writing', tf_filename)


def csv_to_tfrecord(filepath,tf_filename):
    writer = tf.python_io.TFRecordWriter(tf_filename)
    np_data = pd.read_csv(filepath).values
    for row in np_data:
        #print('r')
        example = tf.train.Example(
                features=tf.train.Features(feature={
                    'x': _float_feature(row)
                })
            )
        writer.write(example.SerializeToString())
        
def csv_to_tfrecord_optimized(filepath,tf_filename):
    writer = tf.python_io.TFRecordWriter(tf_filename)
    with open(filepath,'r') as f:
        #f.readline() # skip first line
        for line in f:
            strlist = line.rstrip().split(',')
            flist = [float(i) for i in strlist]
            feature = {
                'x': _float_feature(flist)}
            features=tf.train.Features(feature=feature)
            example = tf.train.Example(features =features)
            writer.write(example.SerializeToString())
            
csv_to_tfrecord_optimized(filepath,tf_filename)

Reading data/creditcard1_8m_test.csv
Writing data/creditcard1_8m_test.tfrecords


In [119]:
features = {'x': tf.FixedLenFeature([29], tf.float32)}
data = []
for s_example in tf.python_io.tf_record_iterator(tf_filename):
    
    example = tf.parse_single_example(s_example, features=features)
    
    data.append(tf.expand_dims(example['x'], 0))
    #print(s_example)


In [120]:
data = tf.concat(data,0)

## Test the tfrecord with the model

In [149]:
# example_batch = tf.train.shuffle_batch(
#         [data], batch_size=10, num_threads=1,
#         capacity=2000, enqueue_many=True,
#         # Ensures a minimum amount of shuffling of examples.
#         min_after_dequeue=2, name='data/creditcard29_test.tfrecords')
# with tf.Session() as sess:
#     init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
#     sess.run(init_op)
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)
#     #TODO: Add train shuffle batch
#     df = sess.run([example_batch])
#     print(df)
# sess = tf.Session()
# with sess.as_default():  
#     l = example['x'].eval()
#     print(example['x'].eval())
# payload = json.dumps({"instances": df[0].tolist()})
# paylaod

In [60]:
filenames = 'data/creditcard1_8m_test.tfrecords'
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV1 shapes: (), types: tf.string>

In [68]:
#for raw_record in raw_dataset.take(1):
for raw_record in tf.python_io.tf_record_iterator(tf_filename):
    example = tf.train.Example()
    example.ParseFromString(raw_record)
    #print(example)
    
example_feature = MessageToDict(example.features)['feature']
encoded_image = example_feature['x']['floatList']['value']
instance = [encoded_image]
payload=  json.dumps({"instances": instance})
#payload

In [71]:
len(encoded_image)

2108909

In [73]:
import requests

import json
# payload = '{"instances": [[1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0], \
# [1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0]]}'
headers = {"content-type": "application/json"}
url = 'http://localhost:8501/v1/models/autoencodermodel:predict'

r = requests.post(url=url, data=payload,headers=headers)
r.text

# duplicate

In [15]:
%%bash


for i in {0..9}; do
    aws s3 cp s3://demo-saeed/iris/iris.txt s3://demo-saeed/iris/iris_${i}.txt 
done

copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris_0.txt
copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris_1.txt
copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris_2.txt
copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris_3.txt
copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris_4.txt
copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris_5.txt
copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris_6.txt
copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris_7.txt
copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris_8.txt
copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris_9.txt


In [16]:
!aws s3 ls s3://demo-saeed/iris/

2019-04-08 18:51:33          0 
2019-04-08 18:51:51       4821 iris.txt
2020-03-07 22:29:27       4821 iris2.txt
2020-03-07 23:54:58       4821 iris_0.txt
2020-03-07 23:54:59       4821 iris_1.txt
2020-03-07 23:54:59       4821 iris_2.txt
2020-03-07 23:55:00       4821 iris_3.txt
2020-03-07 23:55:00       4821 iris_4.txt
2020-03-07 23:55:01       4821 iris_5.txt
2020-03-07 23:55:01       4821 iris_6.txt
2020-03-07 23:55:02       4821 iris_7.txt
2020-03-07 23:55:02       4821 iris_8.txt
2020-03-07 23:55:03       4821 iris_9.txt


In [ ]:
# Start a transform job and wait for it to finish
transformer.transform(batch_input_s3, content_type='text/csv')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()

In [2]:
!aws s3 cp s3://demo-saeed/iris/iris.txt s3://demo-saeed/iris/iris2.txt 

copy: s3://demo-saeed/iris/iris.txt to s3://demo-saeed/iris/iris2.txt


In [18]:
maindf.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [15]:
b.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [167]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'data/train.csv')).upload_file('data/creditcard.csv')


In [168]:
# train_input = sagemaker_session.upload_data('data', key_prefix="{}/{}/{}".format(bucket_name, prefix, 'data') )
# train_input

In [169]:
data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'data',dataset_name)
s3_input_train = sagemaker.s3_input(s3_data = data_location, content_type='csv')
s3_input_train.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://demo-saeed/fraudcredit-keras/data',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}

In [170]:
train_input = 's3://demo-saeed/fraudcredit-keras/data/train.csv'
train_input

's3://demo-saeed/fraudcredit-keras/data/train.csv'

In [56]:
%%bash
# Randomly sample the iris dataset 10 times, then split X and Y
mkdir -p batch_data/XY batch_data/X batch_data/Y
for i in {0..9}; do
    cat data/iris.csv | shuf -n 100 > batch_data/XY/iris_sample_${i}.csv
    cat batch_data/XY/iris_sample_${i}.csv | cut -d',' -f2- > batch_data/X/iris_sample_X_${i}.csv
    cat batch_data/XY/iris_sample_${i}.csv | cut -d',' -f1 > batch_data/Y/iris_sample_Y_${i}.csv
done

OSError: [Errno 12] Cannot allocate memory

In [171]:
import pandas as pd
df = pd.read_csv(train_input, sep=',',  nrows=1000)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
